<a href="https://colab.research.google.com/github/hojin99/aitest/blob/main/%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 데이터 준비

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

### 그리드 서치
* 하이퍼파라미터 튜닝

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0005, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0005, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_)

{'mean_fit_time': array([0.01824312, 0.0091538 , 0.01191521, 0.00797429, 0.01111341]), 'std_fit_time': array([0.00502965, 0.00320816, 0.00617225, 0.00173118, 0.00584024]), 'mean_score_time': array([0.00110669, 0.0009737 , 0.00094595, 0.00092998, 0.00095854]), 'std_score_time': array([1.24000529e-04, 5.91070385e-05, 1.02295808e-04, 6.96183231e-05,
       5.68306280e-05]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0005, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0005}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86538462, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86923077, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

In [ ]:
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0005, 0.0005],
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0005, 0.0005],
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 11, 'min_impurity_decrease': 0.0005, 'min_samples_split': 12}


In [ ]:
import numpy as np

np.max(gs.cv_results_['mean_test_score'])

0.8681935292811135

### 랜덤 서치
* 작업량을 줄이면서 비슷한 결과를 내기 위함

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([5, 5, 4, 7, 9, 8, 0, 2, 7, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 91,  96,  96,  98, 106, 107,  95, 104,  97, 110]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.63248513, 0.12716566, 0.90180529, 0.98200344, 0.0447972 ,
       0.84992539, 0.50767012, 0.42469983, 0.4461504 , 0.95723147])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)         

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6da679b090>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6da65216d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6da668f950>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f6da679bb10>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86
